# Tutorial: Implementing Custom Backward Pass for Non-Native Keras Operators with Jacobinet

## Introduction
This tutorial demonstrates how to implement custom backward passes for non-native Keras operators using Jacobinet. We'll explore two types of backward layers:

- BackwardLinearLayer: For layers with a constant partial derivative.
- BackwardNonLinearLayer: For layers where the partial derivative depends on the input.

## Overview of Custom Operators:
- PlusConstant: A linear layer that adds a constant to the input.
- Clip: A non-linear layer that clips input values to a specified range.

- When running this notebook on Colab, we need to install *decomon* if on Colab. 
- If you run this notebook locally, do it inside the environment in which you [installed *jacobinet*](https://ducoffeM.github.io/jacobinet/main/install.html).

In [ ]:
# On Colab: install the library
on_colab = "google.colab" in str(get_ipython())
if on_colab:
    import sys  # noqa: avoid having this import removed by pycln

    # install dev version for dev doc, or release version for release doc
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install git+https://github.com/ducoffeM/jacobinet@main#egg=decomon
    # install desired backend (by default torch)
    !{sys.executable} -m pip install "torch"
    !{sys.executable} -m pip install "keras"

    # extra librabry used in this notebook
    !{sys.executable} -m pip install "numpy"

In [ ]:
# Set this environment variable *before* importing torch, otherwise it has no effect.
# Ideally, we'd only set this if torch.backends.mps.is_available() is True,
# but checking that requires importing torch first, which would make this setting too late.
# So we preemptively enable the MPS fallback just in case MPS is available.
import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

## 1. Defining the Custom Operator: PlusConstant
We start by creating a custom Keras layer that adds a constant value to its input.

In [ ]:
import keras
import numpy as np
from keras.layers import Layer


class PlusConstant(Layer):
    def __init__(self, constant, **kwargs):
        super(PlusConstant, self).__init__(**kwargs)
        self.constant = keras.ops.convert_to_tensor(constant)

    def call(self, inputs_):
        return inputs_ + self.constant

    def get_config(self):
        config = super().get_config()
        config.update({"constant": keras.saving.serialize_keras_object(self.constant)})
        return config

    @classmethod
    def from_config(cls, config):
        constant_config = config.pop("constant")
        constant = keras.saving.deserialize_keras_object(constant_config)
        return cls(constant=constant, **config)

### Using PlusConstant in a Model:

In [ ]:
from keras.layers import Activation, Dense, Input
from keras.models import Sequential

layers = [Input((10,)), Dense(2), PlusConstant(2), Activation("sigmoid"), Dense(1)]
model_plusconstant = Sequential(layers)

If we try to build a Jacobinet backward model directly, it will fail because PlusConstant is not a native Keras layer:

In [ ]:
from jacobinet import clone_to_backward

try:
    backward_model = clone_to_backward(model_plusconstant)
except ValueError as error:
    print(error)

# 2. Creating BackwardLinearLayer for PlusConstant

Derivative Explanation:
Since $\frac{\partial \text{PlusConstant}(x)}{\partial x}= 1$ the gradient is constant.

Then its backward mapping will be what we denote a BackwardLinearLayer: a backward layer that outputs constant value.
Among existing implemented BackwardLinear layers, we can enumerate Dense(activation=None/linear), Conv(activation=None/linear), AveragePooling, BacthNormalization, Cropping, Padding ...

We only need to override a routine of the call function denoted 

```python
def call_on_reshaped_gradient(
        self, gradient, input=None, training=None, mask=None
    ):
```

this function tales as input the current jacobian (*gradient*) propagated from the output of the neural network
and the input of the layer. This input is not used to compute the backward pass over a BackwardLinear Layer.

## Implementation:

In [ ]:
from jacobinet.layers.layer import BackwardLinearLayer


class BackwardPlusConstant(BackwardLinearLayer):
    def call_on_reshaped_gradient(self, gradient, input=None, training=None, mask=None):
        return gradient

In [ ]:
mapping_keras2backward_classes = {PlusConstant: BackwardPlusConstant}
backward_model_plusconst = clone_to_backward(
    model=model_plusconstant,
    mapping_keras2backward_classes=mapping_keras2backward_classes,
    gradient=keras.Variable(np.ones((1, 1))),
)

# Testing with random input
random_input = np.random.rand(10)
grad_v0 = backward_model_plusconst(random_input[None])
print(grad_v0)

# 3. Enhancing with LayerBackward Property (Optional)

In [ ]:
from keras.layers import Identity, Input


class BackwardPlusConstant_withLayerBackward(BackwardLinearLayer):
    def __init__(self, layer: PlusConstant, **kwargs):
        super().__init__(layer=layer, **kwargs)
        self.layer_backward = Identity()
        self.layer_backward(Input(self.output_dim_wo_batch))


mapping_keras2backward_classes = {PlusConstant: BackwardPlusConstant_withLayerBackward}
backward_model_plusconst_v2 = clone_to_backward(
    model=model_plusconstant,
    mapping_keras2backward_classes=mapping_keras2backward_classes,
    gradient=keras.Variable(np.ones((1, 1))),
)

grad_v2 = backward_model_plusconst_v2(random_input[None])
print(grad_v0 == grad_v2)

# 4. Defining the Custom Operator: Clip

The Clip operator limits inputs to a specified range.

In [ ]:
class Clip(Layer):
    def __init__(self, vmin=0, vmax=1, **kwargs):
        super(Clip, self).__init__(**kwargs)
        self.vmin = vmin
        self.vmax = vmax

    def call(self, inputs_):
        return keras.ops.clip(inputs_, self.vmin, self.vmax)

    def get_config(self):
        config = super().get_config()
        config.update({"vmin": self.vmin, "vmax": self.vmax})
        return config

## Example

In [ ]:
layer_clip = Clip(0, 2)
print(layer_clip(np.ones((1, 1))))  # Output: 1
print(layer_clip(-np.ones((1, 1))))  # Output: 0

# 5. Creating BackwardNonLinearLayer for Clip

Derivative Explanation:
Derivative is 1 if $v_{min}\leq x \leq v_max$, otherwise 0.

But to compute its derivative we need to know the value of its input. We use another BackwardLayer designed for non linear operators:
BackwardNonLinearLayer

## Implementation:

In [ ]:
layers = [Input((10,)), Dense(2), layer_clip, Activation("sigmoid"), Dense(1)]
model_clip = Sequential(layers)

from jacobinet.layers.layer import BackwardNonLinearLayer


class BackwardClip(BackwardNonLinearLayer):

    def call_on_reshaped_gradient(self, gradient, input=None, training=None, mask=None):
        eps = keras.config.epsilon()
        mask_lower_vmax = keras.ops.sign(self.layer.vmax - input - eps)  # 1 iff input <= vmax
        mask_lower_vmin = keras.ops.sign(input - self.layer.vmin + eps)  # 1 iff input >= vmax

        mask = mask_lower_vmax * mask_lower_vmin  # 1 iff x in [vmin, vmax], 0 else
        return gradient * mask


mapping_keras2backward_classes = {Clip: BackwardClip}
backward_model_clip = clone_to_backward(
    model=model_clip,
    mapping_keras2backward_classes=mapping_keras2backward_classes,
    gradient=keras.Variable(np.ones((1, 1))),
)

grad_v3 = backward_model_clip(random_input[None])
print(grad_v3)

# 6. Conclusion
In this tutorial, we demonstrated how to implement custom backward layers using Jacobinet for non-native Keras operators:

- PlusConstant as an example of a linear backward layer.
 
- Clip as an example of a non-linear backward layer.
                                                                                         
These techniques extend *Jacobinet*’s utility to handle custom operators, improving flexibility for neural network models.